In [1]:
# Libraries
import pandas as pd
import numpy as np
import scipy
from scipy.optimize import minimize, fixed_point, Bounds
from scipy.special import expit, logsumexp

In [2]:
# 0: Data
df = pd.read_csv('rust_data_2020.csv').drop(columns = 'Unnamed: 0')
df = df.sort_values(by = ['bus_id', 'period_id'], ascending = [True, True])

# y_it - action taken for bus i at period t
# x_it - mileage for bus i at period t
df.head(5)

,period_id,y_it,x_it,bus_id
0,0,0,0,0
1,1,0,1,0
2,2,0,3,0
3,3,0,4,0
4,4,0,6,0


In [3]:
# 1: delta_x_it frequencies (conditional on y_it = 0)
df['x_it-1'] = df.groupby(by = ['bus_id'])['x_it'].shift(1)
df['delta_x_it'] = df['x_it'] - df['x_it-1']

# Filter for y_it = 0 (action = 0) and calculate frequency distribution of delta_x_it
df0 = df[df['y_it'] == 0].dropna()
stage1 = df0.groupby(by = ['delta_x_it'])['bus_id'].count().reset_index()
stage1 = stage1.rename(columns = {'bus_id' : 'frequency'})
stage1['probability'] = stage1['frequency'] / len(df0)
stage1 = stage1[['delta_x_it', 'probability']].set_index('delta_x_it')
stage1.loc[len] = [0] # Add a row for delta_x_it = 4 with zero probability
stage1

,probability
delta_x_it,
0.0,0.090605
1.0,0.435601
2.0,0.463061
3.0,0.010733
4.0,0.000000


In [4]:
# 2: Constructing transition matrix (conditional on y_it = 0)
n_grid = 201
tpm = np.zeros((n_grid, n_grid))
# Loop to fill in the transition probability matrix
for x_it in range(n_grid):
  for delta_x_it, prob in stage1['probability'].items():
    index = min(x_it + int(delta_x_it), n_grid - 1)
    tpm[x_it, index] += prob
  # Ensure that rows sum to 1
  tpm[x_it, -1] = 1 - tpm[x_it, : -1].sum()

row_sums = tpm.sum(axis = 1)
assert np.allclose(tpm.sum(axis = 1), 1), 'Rows do not sum up to 1!'
tpm

array([[9.06049624e-02, 4.35600781e-01, 4.63061054e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 9.06049624e-02, 4.35600781e-01, ...,
        0.00000000e+00, 0.00000000e+00, 1.11022302e-16],
       [0.00000000e+00, 0.00000000e+00, 9.06049624e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        9.06049624e-02, 4.35600781e-01, 4.73794257e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 9.06049624e-02, 9.09395038e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [5]:
# 3: EV using Rust's trick
def cost(x, a, b):
    return -(a * x) - (b * x**2)

def utility(x, action, theta):
    a, b, RC = theta
    if action == 0:
      return cost(x, a, b)
    else:
      return cost(0 * x, a, b) - RC

def compute_EV(x, P, theta, beta, tol = 1e-12, max_iters = 10000):
    EV = np.zeros(P.shape[0])
    for _ in range(max_iters):
        u0 = utility(x, 0, theta) + beta * EV
        u1 = utility(x, 1, theta) + beta * EV[0]
        W  = logsumexp(np.vstack([u0, u1]), axis = 0)
        EV_new = P @ W
        if np.max(np.abs(EV_new - EV)) < tol:
            break
        EV = EV_new
    return EV

In [6]:
# 4: Conditional choice probabilities
def choice_probabilities(x, theta, beta, EV):
    u0 = utility(x, 0, theta) + beta * EV
    u1 = utility(x, 1, theta) + beta * EV[0]
    V  = np.vstack([u0, u1])
    V  = V - V.max(axis = 0)  # Normalize utilities for numerical stability
    expV = np.exp(V)
    return (expV / expV.sum(axis = 0)).T

In [7]:
# 5: Log‐likelihood objective
def objective(theta, x, P, beta, tol, X_obs, I_obs):
    EV = compute_EV(x, P, theta, beta, tol)
    Pr = choice_probabilities(x, theta, beta, EV)
    return -np.log(Pr[X_obs, I_obs] + 1e-12).sum()

In [8]:
# 6: Analytic gradient
def gradient(theta, x, P, beta, tol, X_obs, I_obs):
    S = len(x)
    EV = compute_EV(x, P, theta, beta, tol)
    u0 = utility(x, 0, theta) + beta * EV
    u1 = utility(x, 1, theta) + beta * EV[0]

    m    = np.maximum(u0, u1)  # Max utility for numerical stability
    exp0 = np.exp(u0 - m)
    exp1 = np.exp(u1 - m)
    p0   = exp0 / (exp0 + exp1)  # CCP for action 0
    p1   = 1 - p0  # CCP for action 1

    # Derivatives of value functions w.r.t. parameters a, b, and RC
    du = {
        0: (-x, np.zeros_like(x)),      # Derivative w.r.t. 'a'
        1: (-x**2, np.zeros_like(x)),   # Derivative w.r.t. 'b'
        2: (np.zeros_like(x), -np.ones_like(x))  # Derivative w.r.t. 'RC'
    }

    I_mat = np.eye(S)
    P0    = P * p0[None, :]  # Transition matrix component for action 0
    P1e0  = np.outer(P @ p1, I_mat[0])  # Transition matrix component for action 1
    A     = I_mat - beta * (P0 + P1e0)  # Matrix A for solving gradient

    grad = np.zeros(3)  # Gradient for parameters [a, b, RC]

    # Loop over each parameter (0, 1, 2 corresponding to [a, b, RC])
    for k in range(3):
        du0, du1 = du[k]  # Get the derivatives w.r.t. the current parameter

        # Compute the gradient contribution for this parameter
        gk = p0 * du0 + p1 * du1
        dEVk = np.linalg.solve(A, P @ gk)

        # Difference in derivatives plus discounted future value
        delta_k = (du0 - du1) + beta * (dEVk - dEVk[0])

        # Sum the contributions from the observed data
        grad_k = (
            (I_obs == 0) * p1[X_obs] * delta_k[X_obs] -
            (I_obs == 1) * p0[X_obs] * delta_k[X_obs]
        ).sum()

        grad[k] = -grad_k  # Store the gradient for this parameter

    return grad

In [9]:
# 7: Optimization
beta, tol  = 0.975, 1e-12
df_obs     = df.dropna().copy()
df_obs['x_scaled'] = df_obs['x_it-1'] / 100
X_obs      = (df_obs['x_scaled'] * 100).astype(int).values
I_obs      = df_obs['y_it'].astype(int).values
x_grid     = np.arange(n_grid) / 100
theta_init = np.ones(3)  # Initial guess for parameters
bounds_obj = Bounds([0, 0, 0], [np.inf, np.inf, np.inf])

results = minimize(fun = objective, x0 = theta_init,
                   args = (x_grid, tpm, beta, tol, X_obs, I_obs),
                   method = 'trust-constr', jac = gradient,
                   bounds = bounds_obj,
                   options = {'verbose': False, 'maxiter': 10000,
                              'gtol': 1e-12, 'xtol': 1e-12
                             }
                   )

print('Estimates:')
print(f'a: {results.x[0]}, b: {results.x[1]}, c: {results.x[2]}')
print('Gradients:')
print(f'a: {results.grad[0]}, b: {results.grad[1]}, c: {results.grad[2]}')

Estimates:
a: 2.839848891393249, b: 0.0062475603564565475, c: 12.446894581694165
Gradients:
a: -2.2023272094884305e-11, b: 9.681806911743251e-09, c: 1.6703083360880555e-11
